In [69]:
from os import listdir, remove
from os.path import isfile, join
from PIL import Image
import numpy as np
import pandas as pd
import os
import cv2
import collections
from tensorflow.keras import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator, ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Flatten
from sklearn.model_selection import train_test_split

In [70]:
# GENERAL VARS
img_shape = (150, 150)

In [71]:
def read_folders_color(path, start=0, end=100):
    elems = {}
    for file_name in listdir(path):
        file_path = join(path, file_name)
        if not isfile(file_path):
            print(f'... {file_name} ...')
            elems[file_name] = read_paths_cv2_color(file_path, start, end)
    return elems

def read_paths_cv2_color(path, start=0, end=100):
    animals = []
    limit = start
    for file_name in listdir(path):
        file_path = join(path, file_name)
        if limit >= end:
            break
        if not isfile(file_path) or file_name.startswith('.'):
            #print(f'WARN - ignoring {file_name}')
            continue

        try:
            img = img = cv2.imread(file_path)
            img_resized = cv2.resize(img, img_shape, interpolation=cv2.INTER_AREA)
            if any('' in x for x in img_resized):
                print(f'ERROR - {file_path} - empty content')
            animals.append(img_resized)
        except:
            print(f'ERROR - {file_path}')
        limit += 1

    return animals

In [72]:
print('start reading imgs....')
elems = read_folders_color('data/img_train/', end=1000)
print('end reading imgs')
print('data transformed!')

start reading imgs....
... cat ...
/Users/Cunillet/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:24: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
... butterfly ...
... sheep ...
... spider ...
... chicken ...
... horse ...
... human ...
... squirrel ...
... cow ...
... elephant ...
end reading imgs
data transformed!


In [73]:
# define a dict with animal - number
animal_names = {
    'butterfly': 0,
    'cat': 1,
    'chicken': 2,
    'cow': 3,
    'human': 4,
    'elephant': 5,
    'horse': 6,
    'sheep': 7,
    'spider': 8,
    'squirrel': 9,
    '0': 'butterfly',
    '1': 'cat',
    '2': 'chicken',
    '3': 'cow',
    '4': 'human',
    '5': 'elephant',
    '6': 'horse',
    '7': 'sheep',
    '8': 'spider',
    '9': 'squirrel'
}

In [74]:
# create an array with all data to get the train - test
full_array = []
targets = []
for k,v in elems.items():
    subset = []
    for subv in v:
        subset.append(subv / 255)
    full_array = full_array + subset.copy()
    targets = targets + ([animal_names[k]] * len(v))
print('transformed to 0..1')
full_array = np.asarray(full_array)
targets = np.asarray(targets)
print('saved')

transformed to 0..1
saved


In [75]:
X_train, X_test, y_train, y_test = train_test_split(full_array, targets, test_size=0.1, random_state=73)

In [76]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(img_shape[0], img_shape[1], 3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(10, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [81]:
model.fit(X_train, y_train, epochs=7)

Train on 9000 samples
Epoch 1/7
9000/9000 [==============================] - 76s 8ms/sample - loss: 0.0443 - accuracy: 0.9848
Epoch 2/7
9000/9000 [==============================] - 76s 8ms/sample - loss: 0.0305 - accuracy: 0.9906
Epoch 3/7
9000/9000 [==============================] - 81s 9ms/sample - loss: 0.0279 - accuracy: 0.9904
Epoch 4/7
9000/9000 [==============================] - 78s 9ms/sample - loss: 0.0323 - accuracy: 0.9907
Epoch 5/7
9000/9000 [==============================] - 78s 9ms/sample - loss: 0.0394 - accuracy: 0.9887
Epoch 6/7
9000/9000 [==============================] - 77s 9ms/sample - loss: 0.0200 - accuracy: 0.9938
Epoch 7/7
9000/9000 [==============================] - 71s 8ms/sample - loss: 0.0479 - accuracy: 0.9853


In [82]:
##################
# STARTING TESTS #
##################
def test_animals(animal=0):
    #print('start reading imgs....')
    test_elems = read_paths_cv2_color(f'data/img_test/{animal_names[str(animal)]}', start=0, end=100)
    #print('end reading imgs')

    test_array = []
    for elem in test_elems:
        test_array.append(elem / 255)

    test_array = np.asarray(test_array)
    #print('data transformed!')
    results = []
    for pos in range(len(test_array)):
        #print(np.asarray(test).shape)
        res = model.predict_classes(np.asarray([test_array[pos]]))
        counts = np.bincount(res)
        mx = np.argmax(counts)
        results.append(mx)
        #print(f'seems to be... {mx} with a {counts[mx] / sum(counts) * 100}%\tres: {res}')
    print(f'{animal_names[str(animal)]}: {results.count(animal)} out of {len(results)}')

In [83]:
for x in range(10):
    test_animals(x)
print('END')

/Users/Cunillet/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:24: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
butterfly: 69 out of 100
cat: 47 out of 100
chicken: 61 out of 100
cow: 56 out of 100
human: 96 out of 100
elephant: 61 out of 100
horse: 54 out of 100
sheep: 44 out of 100
spider: 64 out of 100
squirrel: 59 out of 100
END


In [84]:
# save the model
model.save_weights('data/checkpoints/animals_model_4')

In [28]:
model.load_weights('data/checkpoints/animals_model')


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x179fd1ed0> and <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x179fd1810>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x17ad83950> and <tensorflow.python.keras.layers.core.Flatten object at 0x179f6aad0>).


ValueError: Shapes (128,) and (512,) are incompatible

In [ ]:
# checkpoint 2
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(img_shape[0], img_shape[1], 3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(10, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=9)

In [ ]:
# checkpoint 1
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(img_shape[0], img_shape[1], 3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(10, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=12)

In [ ]:
# checkpoint 3
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(img_shape[0], img_shape[1], 3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(10, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10)

In [ ]:
# checkpoint 4
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(img_shape[0], img_shape[1], 3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(10, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(),
              metrics=['accuracy'])